In [3]:
import os
pth = '/Users/nhathiep/Desktop/Covid/'
f = 'covid19.ipynb'
os.path.join(pth,f)

'/Users/nhathiep/Desktop/Covid/covid19.ipynb'

<div> <img src="Assets/covid_spikes.jpg" width="150" align ='left' alt="alt_text"> </div> 

# Coronavirus (COVID-19)  
<br>
<br>
<u><b>Datasets:</b></u>  <br>
- <b><font color='red'>Confirmed cases</font></b>: time-series data of cumulative #cases.
Raw data <a href=https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv> here </a><br>
- <b><font color='red'>Latest cases</font></b>: Latest stats on incidence rate, fatality, mortality. 
Raw data <a href =https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-02-2021.csv> here </a>

<u><b>Source:</b></u>  <br>
- Coronavirus Case Data is provided by <a href='https://github.com/CSSEGISandData/COVID-19'>Johns Hopkins University</a>
- Field definition: [Link here](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data)

<u><b>Benefits/Automation:</b></u>  <br>
- The data is automatically downloaded from the source and the latest report is systematically identified<br>
- Data wrangling (reformatting & pivoting) and new calculations are automated<br>
- The data can be transferred onto an Excel file and said Excel file can be updated programmatically (using Python)

# Coronavirus (COVID-19)

<font face = "Verdana" size ="3">Coronavirus is a family of viruses that are named after their spiky crown. The novel coronavirus, also known as SARS-CoV-2, is a contagious respiratory virus that first reported in Wuhan, China. On 2/11/2020, the World Health Organization designated the name COVID-19 for the disease caused by the novel coronavirus. This notebook aims at exploring COVID-19 through data analysis and projections. 
 
   Coronavirus Case Data is provided by <a href='https://github.com/CSSEGISandData/COVID-19'>Johns Hopkins University</a>
   <br>Mobility data is provided by <a href='https://www.apple.com/covid19/mobility'>Apple</a>
   <br>Learn more from the <a href='https://www.who.int/emergencies/diseases/novel-coronavirus-2019'>World Health Organization</a>
   <br>Learn more from the <a href='https://www.cdc.gov/coronavirus/2019-ncov'>Centers for Disease Control and Prevention</a>
   <br>Check out map visualizations from  <a href='https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6'>JHU CCSE Dashboard</a>
   <br>Source code is on <a href='https://github.com/therealcyberlord'>my Github</a>
  
   
   <font face = "Verdana" size ="3">
   Feel free to provide me with feedbacks. 
       <br> Last update: 1/3/2021 6:39 PM ET. Updates to the data table. 
       <br> New Updates: Daily report data update for 1/3. time series data update for 1/2. Mobility data update for 1/2. 
   </font>
       <br>
   <font face = "Verdana" size ="3"> *Prediction models are trained from a later starting date (3/13/2020). Therefore, earlier dates might not be accurate. </font>
   <font face = "Verdana" size ="1">
    
    </font>
 <font face = "Verdana" size ="3"> Keep strong, world. We can get through this! </font>
   <font face = "Verdana" size = "3"> Thank you guys for all the support. </font>
<br>
<font face = "Verdana" size ="3"> Sections </font>
* <a href='#world_wide_graphs'>Exploring Global Coronavirus Cases</a>
* <a href='#country_graphs'>Exploring Coronavirus Cases From Different Countries</a>
* <a href='#prediction'>Worldwide Confirmed Cases Prediction</a>
* <a href='#data_table'>Data Table</a>
* <a href='#pie_charts'>Pie Charts</a>
* <a href='#bar_charts'>Bar Charts</a>
* <a href='#hospital_testing'>US Testing Data</a>
* <a href='#mobility_data'>Mobility Data for Hotspots</a>

In [1]:
#!conda install  scikit-learn --yes

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime as dt
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import warnings
warnings.filterwarnings("ignore")

Import the data (make sure you update this on a daily basis)

In [2]:
#Global Data - All countries time-series
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
#Daily reports on 2n Jan - for Global and US
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-02-2021.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/01-02-2021.csv')
#Apple Data
#apple_mobility = pd.read_csv('https://covid19-static.cdn-apple.com/covid19-mobility-data/2023HotfixDev27/v3/en-us/applemobilitytrends-2021-01-02.csv')

In [3]:
#Today
datefile = dt.datetime.today()
datefile.strftime("%m-%d-%y")

'12-21-21'

In [4]:
#Looking for the most recent files for Latest Y& US_medical
count = 0
while count < 10:
    try: 
        datefile = dt.datetime.today()- dt.timedelta(days= count)
        datetext = datefile.strftime("%m-%d-%Y")
        latest_data = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{datetext}.csv')
        us_medical_data = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/{datetext}.csv')     
        break
    except:
        count += 1
        print(datetext)            
    

12-21-2021
12-20-2021


In [5]:
latest_data.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-12-20 04:21:43,33.93911,67.709953,157787,7335,NaN,NaN,Afghanistan,405.326803,4.648672
1,NaN,NaN,NaN,Albania,2021-12-20 04:21:43,41.15330,20.168300,205777,3166,NaN,NaN,Albania,7150.496907,1.538559
2,NaN,NaN,NaN,Algeria,2021-12-20 04:21:43,28.03390,1.659600,214592,6184,NaN,NaN,Algeria,489.365783,2.881748
3,NaN,NaN,NaN,Andorra,2021-12-20 04:21:43,42.50630,1.521800,20549,134,NaN,NaN,Andorra,26595.483078,0.652100
4,NaN,NaN,NaN,Angola,2021-12-20 04:21:43,-11.20270,17.873900,65938,1738,NaN,NaN,Angola,200.625152,2.635809


In [6]:
#Incidence rate
#latest_data.iloc[[9]]
#latest_data.set_index('Province_State')
latest_data.iloc[[9]]
#['Incident_Rate']

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
9,NaN,NaN,New South Wales,Australia,2021-12-20 04:21:43,-33.8688,151.2093,100040,643,NaN,NaN,"New South Wales, Australia",1232.323232,0.642743


In [7]:
latest_data.set_index('Province_State', inplace = True)

In [8]:
latest_data.dtypes

FIPS                   float64
Admin2                  object
Country_Region          object
Last_Update             object
Lat                    float64
Long_                  float64
Confirmed                int64
Deaths                   int64
Recovered              float64
Active                 float64
Combined_Key            object
Incident_Rate          float64
Case_Fatality_Ratio    float64
dtype: object

In [9]:
latest_data.loc[['New South Wales']]

,FIPS,Admin2,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
Province_State,,,,,,,,,,,,,
New South Wales,NaN,NaN,Australia,2021-12-20 04:21:43,-33.8688,151.2093,100040,643,NaN,NaN,"New South Wales, Australia",1232.323232,0.642743


In [10]:
#Underlying population for New South Wales
#Incidence rate is cases per 100,000 persons
incidence_rate = latest_data.loc['New South Wales']['Incident_Rate']/100000
NSW_pop = latest_data.loc['New South Wales']['Confirmed'] / incidence_rate
print(NSW_pop)
print(f'{NSW_pop:,.0f} inhabitants in NSW') # the comma means to display with thousand separator and '.0f' means no decimal 

8117999.999999998
8,118,000 inhabitants in NSW


In [11]:
#Checking the fatality rate
#definition: https://www.cdc.gov/csels/dsepd/ss1978/lesson3/section3.html
#Fatality rate is deaths per 100 persons (%)
NSW_deaths = latest_data.loc['New South Wales']['Deaths'] 
NSW_confirmed = latest_data.loc['New South Wales']['Confirmed'] 
NSW_CaseFatalityRatio = NSW_deaths/NSW_confirmed
print(f'{NSW_CaseFatalityRatio:0%} ')

#Check it is the same number as table
#Multiply by 100 to convert into percent and check with 14 decimals accuracy using round(number,14)
check = round(NSW_CaseFatalityRatio*100, 14) == round(latest_data.loc['New South Wales']['Case_Fatality_Ratio'],14)
#print(latest_data.loc['New South Wales']['Case_Fatality_Ratio'] )
#print(round(NSW_CaseFatalityRatio*100,14) )

print (f'Calculated Fatality ratio is equal to Downloaded Fatality ratio: \033[1m {check}') #To print in Bold \033[1m

0.642743% 
Calculated Fatality ratio is equal to Downloaded Fatality ratio:  True


In [12]:
latest_data.reset_index(inplace = True)

In [13]:
#confirmed cases
confirmed_df[confirmed_df['Province/State']=='New South Wales']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/10/21,12/11/21,12/12/21,12/13/21,12/14/21,12/15/21,12/16/21,12/17/21,12/18/21,12/19/21
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,85447,85924,86453,87248,88595,90333,92532,95000,97558,100040


# Bokeh

In [14]:
import numpy as np # we will use this later, so import it n
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.widgets import Slider
from bokeh.models import DateRangeSlider

#In this case, we are in the Jupyter notebook, so we will call output_notebook() below. We only need to call this once, and all subsequent calls to show() will display inline in the notebook.
output_notebook()

Loading BokehJS ...

# Confirmed cases - what the data looks like (as downloaded from raw file)
Filtered for France
![alt text](Assets/Confirmed_cases.png)

## Showing other countries

In [31]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/10/21,12/11/21,12/12/21,12/13/21,12/14/21,12/15/21,12/16/21,12/17/21,12/18/21,12/19/21
0,Other,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,157858,157858,157858,157648,157660,157665,157725,157734,157745,157787
1,Other,Albania,41.15330,20.168300,0,0,0,0,0,0,...,203215,203524,203787,203925,204301,204627,204928,205224,205549,205777
2,Other,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,212434,212652,212848,213058,213288,213533,213745,214044,214330,214592
3,Other,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,19440,19440,19440,19440,20136,20136,20549,20549,20549,20549
4,Other,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,65371,65397,65404,65404,65431,65565,65648,65760,65868,65938


# New cases

In [16]:
confirmed_df['Province/State']= confirmed_df['Province/State'].fillna('Other')

In [17]:
confirmed_df[confirmed_df['Country/Region']=='Brazil']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/10/21,12/11/21,12/12/21,12/13/21,12/14/21,12/15/21,12/16/21,12/17/21,12/18/21,12/19/21
30,Other,Brazil,-14.235,-51.9253,0,0,0,0,0,0,...,22177059,22177059,22177059,22177059,22195775,22201221,22201221,22204941,22204941,22213762


In [18]:
confirmed_df[confirmed_df['Country/Region']=='France']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/10/21,12/11/21,12/12/21,12/13/21,12/14/21,12/15/21,12/16/21,12/17/21,12/18/21,12/19/21
119,French Guiana,France,3.933900,-53.125800,0,0,0,0,0,0,...,46304,46304,46304,46421,46421,46510,46590,46690,46690,46690
120,French Polynesia,France,-17.679700,149.406800,0,0,0,0,0,0,...,46334,46334,46334,46334,46334,46335,46342,46342,46342,46342
121,Guadeloupe,France,16.265000,-61.551000,0,0,0,0,0,0,...,55080,55080,55080,55375,55375,55375,55375,55375,55375,55375
122,Martinique,France,14.641500,-61.024200,0,0,0,0,0,0,...,46251,46251,46251,46410,46410,46597,46827,46827,46827,46827
123,Mayotte,France,-12.827500,45.166244,0,0,0,0,0,0,...,21088,21088,21088,21088,21129,21139,21139,21180,21180,21180
124,New Caledonia,France,-20.904305,165.618042,0,0,0,0,0,0,...,12479,12479,12479,12515,12525,12525,12557,12581,12581,12581
125,Reunion,France,-21.115100,55.536400,0,0,0,0,0,0,...,63863,63863,63863,63863,67237,67237,67237,67237,67237,67237
126,Saint Barthelemy,France,17.900000,-62.833300,0,0,0,0,0,0,...,1674,1674,1674,1683,1683,1683,1683,1683,1683,1683
127,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,...,94,94,94,95,96,96,96,96,96,96
128,St Martin,France,18.070800,-63.050100,0,0,0,0,0,0,...,4040,4040,4040,4059,4059,4059,4059,4059,4059,4059


In [19]:
confirmed_df['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guin

In [20]:
confirmed_cases = confirmed_df.groupby(['Province/State', 'Country/Region', 'Lat', 'Long'])[confirmed_df.columns[4:]].sum().stack()

In [21]:
#confirmed_cases.rename({'level4': 'Date', '0': 'Cases'}, axis='columns', inplace = True)
confirmed_cases = confirmed_cases.reset_index()
confirmed_cases

,Province/State,Country/Region,Lat,Long,level_4,0
0,Alberta,Canada,53.9333,-116.5765,1/22/20,0
1,Alberta,Canada,53.9333,-116.5765,1/23/20,0
2,Alberta,Canada,53.9333,-116.5765,1/24/20,0
3,Alberta,Canada,53.9333,-116.5765,1/25/20,0
4,Alberta,Canada,53.9333,-116.5765,1/26/20,0
...,...,...,...,...,...,...
194039,Zhejiang,China,29.1832,120.0934,12/15/21,1823
194040,Zhejiang,China,29.1832,120.0934,12/16/21,1867
194041,Zhejiang,China,29.1832,120.0934,12/17/21,1944
194042,Zhejiang,China,29.1832,120.0934,12/18/21,1975


In [22]:
confirmed_cases.columns[5]

0

In [23]:
confirmed_cases.rename({'level_4': 'Date', confirmed_cases.columns[5]: 'Cases'}, axis='columns', inplace = True)
#confirmed_cases.columns = confirmed_cases.columns[:5]
confirmed_cases

,Province/State,Country/Region,Lat,Long,Date,Cases
0,Alberta,Canada,53.9333,-116.5765,1/22/20,0
1,Alberta,Canada,53.9333,-116.5765,1/23/20,0
2,Alberta,Canada,53.9333,-116.5765,1/24/20,0
3,Alberta,Canada,53.9333,-116.5765,1/25/20,0
4,Alberta,Canada,53.9333,-116.5765,1/26/20,0
...,...,...,...,...,...,...
194039,Zhejiang,China,29.1832,120.0934,12/15/21,1823
194040,Zhejiang,China,29.1832,120.0934,12/16/21,1867
194041,Zhejiang,China,29.1832,120.0934,12/17/21,1944
194042,Zhejiang,China,29.1832,120.0934,12/18/21,1975


In [24]:
by_group = confirmed_cases.groupby(['Province/State','Country/Region'])
confirmed_cases['Newcases'] = list(by_group.apply(lambda x: x['Cases']-x['Cases'].shift()))
confirmed_cases['Newcases'].fillna(0, inplace = True)

confirmed_cases['Date'] = confirmed_cases['Date'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%y') )


confirmed_cases[690:700]

#for i in confirmed_cases[550:600].iterrows(): print(i)

,Province/State,Country/Region,Lat,Long,Date,Cases,Newcases
690,Alberta,Canada,53.9333,-116.5765,2021-12-12,338428,0.0
691,Alberta,Canada,53.9333,-116.5765,2021-12-13,339291,863.0
692,Alberta,Canada,53.9333,-116.5765,2021-12-14,339541,250.0
693,Alberta,Canada,53.9333,-116.5765,2021-12-15,339997,456.0
694,Alberta,Canada,53.9333,-116.5765,2021-12-16,340470,473.0
695,Alberta,Canada,53.9333,-116.5765,2021-12-17,341023,553.0
696,Alberta,Canada,53.9333,-116.5765,2021-12-18,341023,0.0
697,Alberta,Canada,53.9333,-116.5765,2021-12-19,341023,0.0
698,Anguilla,United Kingdom,18.2206,-63.0686,2020-01-22,0,0.0
699,Anguilla,United Kingdom,18.2206,-63.0686,2020-01-23,0,0.0


# Confirmed cases - Pivoting the data (France)
- **Data wrangling**: dates are pivoted in such a way that they are now a dimension (as opposed to columns before)
- **New calculation**: number of new cases is added

In [30]:
confirmed_cases[(confirmed_cases['Country/Region']== 'France') & (confirmed_cases['Province/State']== 'Other')]

,Province/State,Country/Region,Lat,Long,Date,Cases,Newcases
80968,Other,France,46.2276,2.2137,2020-01-22,0,0.0
80969,Other,France,46.2276,2.2137,2020-01-23,0,0.0
80970,Other,France,46.2276,2.2137,2020-01-24,2,2.0
80971,Other,France,46.2276,2.2137,2020-01-25,3,1.0
80972,Other,France,46.2276,2.2137,2020-01-26,3,0.0
...,...,...,...,...,...,...,...
81661,Other,France,46.2276,2.2137,2021-12-15,8202064,65427.0
81662,Other,France,46.2276,2.2137,2021-12-16,8262620,60556.0
81663,Other,France,46.2276,2.2137,2021-12-17,8320607,57987.0
81664,Other,France,46.2276,2.2137,2021-12-18,8379143,58536.0


**Notes**  
Cases: means cumulative cases  
New cases: means new daily cases (the delta between Cases on a given day and Cases on the day before)

In [26]:
len(confirmed_cases)

194044

# Confirmed cases - France
**New Cases**
![alt text](Assets/France_newcases.png)

**Note**: 5 waves

**Cumulative Cases**
![alt text](Assets/France_cumulcases.png)

# Confirmed cases - Pivoting the data (NSW)

In [27]:
z = confirmed_cases[confirmed_cases['Province/State']=='New South Wales']
x = z['Date']
x_values = [dt.datetime.strftime(x,'%m/%d/%y') for x in x]
y = z['Newcases']
z

,Province/State,Country/Region,Lat,Long,Date,Cases,Newcases
34202,New South Wales,Australia,-33.8688,151.2093,2020-01-22,0,0.0
34203,New South Wales,Australia,-33.8688,151.2093,2020-01-23,0,0.0
34204,New South Wales,Australia,-33.8688,151.2093,2020-01-24,0,0.0
34205,New South Wales,Australia,-33.8688,151.2093,2020-01-25,0,0.0
34206,New South Wales,Australia,-33.8688,151.2093,2020-01-26,3,3.0
...,...,...,...,...,...,...,...
34895,New South Wales,Australia,-33.8688,151.2093,2021-12-15,90333,1738.0
34896,New South Wales,Australia,-33.8688,151.2093,2021-12-16,92532,2199.0
34897,New South Wales,Australia,-33.8688,151.2093,2021-12-17,95000,2468.0
34898,New South Wales,Australia,-33.8688,151.2093,2021-12-18,97558,2558.0


# Confirmed cases - Australia, NSW
**New Cases**
![alt text](Assets/NSW_newcases.png)

**Note**: 3 waves - 1st wave in Apr20 (original sars-cov-2), 2nd wave Delta variant Aug21, 3rd wave Dec21 as all restrictions are lifted (due to vaccination rates exceeding target of 80%)

**Cumulative Cases**
![alt text](Assets/NSW_cumulcases.png)

# Latest Data - what the data Looks like  (as downloaded from raw file)

In [222]:
latest_data[latest_data.Country_Region =='Australia']

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
8,NaN,NaN,Australian Capital Territory,Australia,2021-12-20 04:21:43,-35.4735,149.0124,2291,15,NaN,NaN,"Australian Capital Territory, Australia",535.155338,0.654736
9,NaN,NaN,New South Wales,Australia,2021-12-20 04:21:43,-33.8688,151.2093,100040,643,NaN,NaN,"New South Wales, Australia",1232.323232,0.642743
10,NaN,NaN,Northern Territory,Australia,2021-12-20 04:21:43,-12.4634,130.8456,360,1,NaN,NaN,"Northern Territory, Australia",146.579805,0.277778
11,NaN,NaN,Queensland,Australia,2021-12-20 04:21:43,-27.4698,153.0251,2356,7,NaN,NaN,"Queensland, Australia",46.056104,0.297114
12,NaN,NaN,South Australia,Australia,2021-12-20 04:21:43,-34.9285,138.6007,1401,4,NaN,NaN,"South Australia, Australia",79.760888,0.285510
13,NaN,NaN,Tasmania,Australia,2021-12-20 04:21:43,-42.8821,147.3272,250,13,NaN,NaN,"Tasmania, Australia",46.685341,5.200000
14,NaN,NaN,Victoria,Australia,2021-12-20 04:21:43,-37.8136,144.9631,146859,1454,NaN,NaN,"Victoria, Australia",2215.101284,0.990065
15,NaN,NaN,Western Australia,Australia,2021-12-20 04:21:43,-31.9505,115.8605,1125,9,NaN,NaN,"Western Australia, Australia",42.765909,0.800000


Field definition: [Link here](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data)  
**Note**: "Confirmed" is the latest cumulative case count

In [223]:
#Focusing on NSW
latest_data[latest_data.Province_State =='New South Wales']

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
9,NaN,NaN,New South Wales,Australia,2021-12-20 04:21:43,-33.8688,151.2093,100040,643,NaN,NaN,"New South Wales, Australia",1232.323232,0.642743


# Ranking of countries

In [250]:
latest_data[latest_data['Country_Region'] == 'Australia']

,FIPS,Admin2,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
Province_State,,,,,,,,,,,,,
Australian Capital Territory,NaN,NaN,Australia,2021-12-20 04:21:43,-35.4735,149.0124,2291,15,NaN,NaN,"Australian Capital Territory, Australia",535.155338,0.654736
New South Wales,NaN,NaN,Australia,2021-12-20 04:21:43,-33.8688,151.2093,100040,643,NaN,NaN,"New South Wales, Australia",1232.323232,0.642743
Northern Territory,NaN,NaN,Australia,2021-12-20 04:21:43,-12.4634,130.8456,360,1,NaN,NaN,"Northern Territory, Australia",146.579805,0.277778
Queensland,NaN,NaN,Australia,2021-12-20 04:21:43,-27.4698,153.0251,2356,7,NaN,NaN,"Queensland, Australia",46.056104,0.297114
South Australia,NaN,NaN,Australia,2021-12-20 04:21:43,-34.9285,138.6007,1401,4,NaN,NaN,"South Australia, Australia",79.760888,0.285510
Tasmania,NaN,NaN,Australia,2021-12-20 04:21:43,-42.8821,147.3272,250,13,NaN,NaN,"Tasmania, Australia",46.685341,5.200000
Victoria,NaN,NaN,Australia,2021-12-20 04:21:43,-37.8136,144.9631,146859,1454,NaN,NaN,"Victoria, Australia",2215.101284,0.990065
Western Australia,NaN,NaN,Australia,2021-12-20 04:21:43,-31.9505,115.8605,1125,9,NaN,NaN,"Western Australia, Australia",42.765909,0.800000


In [251]:
latest_data['Population'] = latest_data['Confirmed']/(latest_data['Incident_Rate']/100000)
latest_data['Incidence'] = (latest_data['Population']/100000)*(latest_data['Incident_Rate'])

In [252]:
latest_data.head()

,FIPS,Admin2,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Incidence
Province_State,,,,,,,,,,,,,,,
NaN,NaN,NaN,Afghanistan,2021-12-20 04:21:43,33.93911,67.709953,157787,7335,NaN,NaN,Afghanistan,405.326803,4.648672,38928341.0,157787.0
NaN,NaN,NaN,Albania,2021-12-20 04:21:43,41.15330,20.168300,205777,3166,NaN,NaN,Albania,7150.496907,1.538559,2877800.0,205777.0
NaN,NaN,NaN,Algeria,2021-12-20 04:21:43,28.03390,1.659600,214592,6184,NaN,NaN,Algeria,489.365783,2.881748,43851043.0,214592.0
NaN,NaN,NaN,Andorra,2021-12-20 04:21:43,42.50630,1.521800,20549,134,NaN,NaN,Andorra,26595.483078,0.652100,77265.0,20549.0
NaN,NaN,NaN,Angola,2021-12-20 04:21:43,-11.20270,17.873900,65938,1738,NaN,NaN,Angola,200.625152,2.635809,32866268.0,65938.0


In [253]:
latest_data[latest_data.Country_Region =='US']

,FIPS,Admin2,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Incidence
Province_State,,,,,,,,,,,,,,,
Alabama,1001.0,Autauga,US,2021-12-20 04:21:43,32.539527,-86.644082,10669,159,NaN,NaN,"Autauga, Alabama, US",19096.457785,1.490299,55869.0,10669.0
Alabama,1003.0,Baldwin,US,2021-12-20 04:21:43,30.727750,-87.722071,38506,592,NaN,NaN,"Baldwin, Alabama, US",17249.164554,1.537423,223234.0,38506.0
Alabama,1005.0,Barbour,US,2021-12-20 04:21:43,31.868263,-85.387129,3723,81,NaN,NaN,"Barbour, Alabama, US",15081.422669,2.175665,24686.0,3723.0
Alabama,1007.0,Bibb,US,2021-12-20 04:21:43,32.996421,-87.125115,4397,95,NaN,NaN,"Bibb, Alabama, US",19634.723587,2.160564,22394.0,4397.0
Alabama,1009.0,Blount,US,2021-12-20 04:21:43,33.982109,-86.567906,10959,195,NaN,NaN,"Blount, Alabama, US",18951.682634,1.779359,57826.0,10959.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wyoming,56039.0,Teton,US,2021-12-20 04:21:43,43.935225,-110.589080,5478,14,NaN,NaN,"Teton, Wyoming, US",23346.403000,0.255568,23464.0,5478.0
Wyoming,56041.0,Uinta,US,2021-12-20 04:21:43,41.287818,-110.547578,4113,31,NaN,NaN,"Uinta, Wyoming, US",20335.212103,0.753708,20226.0,4113.0
Wyoming,90056.0,Unassigned,US,2021-12-20 04:21:43,NaN,NaN,0,0,NaN,NaN,"Unassigned, Wyoming, US",NaN,NaN,NaN,NaN


In [254]:
countriesrank = latest_data.groupby('Country_Region')[['Population']].sum().sort_values('Population', ascending = False)
countriesrank

,Population
Country_Region,
China,1.404676e+09
India,1.371337e+09
US,3.323034e+08
Indonesia,2.735236e+08
Brazil,2.101471e+08
...,...
Palau,1.800800e+04
Holy See,8.090000e+02
MS Zaandam,0.000000e+00


# Latest Data - aggregation: total by country (summing all provinces/states)

In [264]:
countriesrank = latest_data.groupby('Country_Region')[['Population','Confirmed','Deaths','Recovered','Active']].sum().sort_values('Population', ascending = False)
countriesrank

,Population,Confirmed,Deaths,Recovered,Active
Country_Region,,,,,
China,1.404676e+09,112989,4849,0.0,0.0
India,1.371337e+09,34746838,477554,0.0,0.0
US,3.323034e+08,50846828,806439,0.0,0.0
Indonesia,2.735236e+08,4260544,144002,0.0,0.0
Brazil,2.101471e+08,22213762,617803,0.0,0.0
...,...,...,...,...,...
Palau,1.800800e+04,8,0,0.0,0.0
Holy See,8.090000e+02,27,0,0.0,0.0
MS Zaandam,0.000000e+00,9,2,0.0,0.0


In [265]:
p = countriesrank.loc['Spain']['Population']
f'{p:,.0f} inhabitants'

'46,937,060 inhabitants'

# Latest data - Calculating additional metrics (Population, %Infected, Mortality, Fatality)

In [266]:
countriesrank = countriesrank.loc[countriesrank.index!='Afghanistan'].head(20).sort_values('Population')
countriesrank['%pop_infected'] = countriesrank['Confirmed'] / countriesrank['Population']
countriesrank['Mortality']  = countriesrank['Deaths'] / countriesrank['Population']
countriesrank['Fatality'] = countriesrank['Deaths'] / countriesrank['Confirmed']
countriesrank

,Population,Confirmed,Deaths,Recovered,Active,%pop_infected,Mortality,Fatality
Country_Region,,,,,,,,
Thailand,6.979998e+07,2191528,21377,0.0,0.0,0.031397,0.000306,0.009754
Germany,8.301921e+07,6812746,108356,0.0,0.0,0.082062,0.001305,0.015905
Iran,8.399295e+07,6170979,131083,0.0,0.0,0.073470,0.001561,0.021242
Turkey,8.433907e+07,9173089,80415,0.0,0.0,0.108764,0.000953,0.008766
Congo (Kinshasa),8.956140e+07,64448,1126,0.0,0.0,0.000720,0.000013,0.017471
Vietnam,9.733858e+07,1540478,29566,0.0,0.0,0.015826,0.000304,0.019193
Egypt,1.023344e+08,375330,21361,0.0,0.0,0.003668,0.000209,0.056913
Philippines,1.095811e+08,2837577,50739,0.0,0.0,0.025895,0.000463,0.017881
Ethiopia,1.149636e+08,376375,6866,0.0,0.0,0.003274,0.000060,0.018242


In [267]:
#countriesrank.Deaths.tolist()

In [268]:
#countriesrank.index.tolist()

# Death toll for top-20 most populated countries
Sorted by population size in descending order (China 1st, India 2nd etc.)  
Size of the Bar represents the number of deaths
![alt text](Assets/deaths_ranking.png)

In [270]:
countriesrank.index.tolist()

['Thailand',
 'Germany',
 'Iran',
 'Turkey',
 'Congo (Kinshasa)',
 'Vietnam',
 'Egypt',
 'Philippines',
 'Ethiopia',
 'Japan',
 'Mexico',
 'Russia',
 'Bangladesh',
 'Nigeria',
 'Pakistan',
 'Brazil',
 'Indonesia',
 'US',
 'India',
 'China']

# Bubble chart: % Infected population,Fatality, Death toll
- **x axis** = %population infected (cumulative cases/population), approximation 
- **y axis** = Fatality rate | **bubble size** = # deaths
![alt text](Assets/bubble.png)

### Comments
- % population infected (approximation only as this does not consider the fact that reinfected people inflate confirmed cases): impacted by testing policy (number of test centres, free or paid tests, test kit avaibility etc); and by willingness of countries to report accurate case numbers
- Fatality rate: impacted by vaccination rate, hospital capacity/equipment; and by willingness of countries to report accurate case numbers and death toll
- Deaths: this should be analysed concurrently with population size